## **自動車の評価**
##### _自動車の価値を推定しよう_


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier as DT
from lightgbm import LGBMClassifier as LGBM
from sklearn.model_selection import GridSearchCV

sns.set()

In [ ]:
train_data = pd.read_csv('C:\\Users\\busin\\My_Products\\自動車の評価\\train.tsv', sep='\t')
test_data = pd.read_csv('C:\\Users\\busin\\My_Products\\自動車の評価\\test.tsv', sep='\t')
submit = pd.read_csv('C:\\Users\\busin\\My_Products\\自動車の評価\\sample_submit.csv',  header=None)

In [ ]:
label = LabelEncoder()
train_data['class'] = label.fit_transform(train_data['class'])
train_data

In [ ]:
test_data['class'] = 'test'
test_data

In [ ]:
merged_data = pd.concat([train_data, test_data], axis=0).reset_index(drop=True)
merged_data

In [ ]:
merged_data_dummied = pd.get_dummies(merged_data.drop('class', axis=1))
merged_data_dummied['class'] = merged_data['class']
merged_data_dummied

In [ ]:
X_train = merged_data_dummied[merged_data_dummied["class"] != 'test'].drop(['class','id'], axis=1)
y_train = merged_data_dummied[merged_data_dummied["class"] != 'test']['class'].astype(int)
X_test = merged_data_dummied[merged_data_dummied['class'] == 'test'].drop(['id','class'], axis=1)

In [ ]:
dt = DT(max_depth=10, min_samples_leaf=5, min_samples_split=0.01 )
lgbm = LGBM(objective="binary", learning_rate= 0.01, num_leaves=35, n_estimators=100, max_depth=10)

In [ ]:
def dt_params():
  ret = {
        'max_depth': [1, 5, 10, 50, 100],
        'min_samples_leaf': [1, 5, 10, 50, 100],
        'min_samples_split': [0.01, 0.02, 0.03, 0.04, 0.05, 0.1]
  }
  return ret

def lgbm_params():
  ret = {
        "objective":["binary"],
        "learning_rate": [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0,1],
        "num_leaves": [31, 41, 51, 61, 100],
        "n_estimators":[10, 50, 100],
        "max_depth":[1, 2, 5, 10, 20, 50]
  }
  return ret

estimator = LGBM()
model_params = lgbm_params()

gcv = GridSearchCV(estimator, model_params, cv=5, n_jobs=2)
gcv.fit(X_train, y_train)

In [ ]:
gcv.best_params_

In [ ]:
best = gcv.best_estimator_
y_pred = best.predict(X_test)

In [ ]:
y_pred = label.inverse_transform(y_pred)
submit[1] = y_pred
submit.to_csv('first_submit.csv', header=None, index=False)